In [22]:
import math
import numpy as np
import matplotlib.pyplot as pp
% matplotlib inline

import libstempo

In [33]:
# I didn't have time to study your parameter implementation, so this is my own

class Parameter(object):
    def __init__(self,name):
        self.name = name

# the following two are class *factories*: they return a customized class
# then the actual parameters (with names) are obtained by instantiating the class

def Uniform(pmin,pmax):
    # this naive class definition can be improved with type, etc.
    # but will be good enough for a demo
    class Uniform(Parameter):
        def pdf(self,p):
            return 1.0 / (pmax - pmin) if (pmin < p < pmax) else 0.0
                
        def __repr__(self):
            return '"{}":Uniform({},{})'.format(self.name,pmin,pmax)
        
    return Uniform

def Normal(mu=0,sigma=1):
    class Normal(Parameter):
        norm = 1.0 / math.sqrt(2 * math.pi * sigma**2)
        
        def pdf(self,x):
            return self.norm * math.exp(-0.5*(x - mu)**2/sigma**2)
        
    return Normal

In [16]:
# a uniformly distributed parameter
u = Uniform(0,10)
u

__main__.Uniform.<locals>.Uniform

In [17]:
# instantiations of this parameter
u1 = u('uniform1')
u2 = u('uniform2')

In [18]:
u1

"uniform1":Uniform(0,10)

In [20]:
u1.pdf(5), u1.pdf(-1)

(0.1, 0.0)

In [34]:
n = Normal(2,5)

In [35]:
n1 = n('normal1')
n2 = n('normal2')

In [36]:
n1.pdf(2)

0.07978845608028654

In [37]:
# base signal class. Not much in it yet
class Signal(object):
    @property
    def params(self):
        return self._params.values()

In [38]:
# again, a factory that returns a customized class, reusable to generate
# signal instances for many pulsars

def MeasurementNoise(efac = Uniform(0.5,1.5)):
    class MeasurementNoise(Signal):
        def __init__(self,psr):
            self._psr = psr
            
            self._params = {'efac': efac(psr.name + '_efac')}
            self._ndiag = (psr.toaerrs*1e-6)**2

        # here we assume that we pass all parameters to all the "tensor"
        # methods of the instantiated signals. An alternative is to set
        # a persistent value for each parameter. It's less functional,
        # but (often) practicality beats purity
        def ndiag(self,params):
            return params[self._params['efac'].name]**2 * self._ndiag
        
    return MeasurementNoise

In [ ]:
def EquadNoise(log10_equad = Uniform(-18,-14)):
    class EquadNoise(Signal):
        def __init__(self,psr):
            self._psr = psr
            
            self._params = {'log10_equad': log10_equad(psr.name + '_log10_equad')}
            self._ndiag = np.ones_like(psr.toaerrs)

        def ndiag(self,params):
            return 10.0**(2*params[self._params['log10_equad'].name]) * self._ndiag
        
    return EquadNoise

In [39]:
w = MeasurementNoise()

In [40]:
psr = libstempo.tempopulsar('../2222.par','../2222.tim')

In [41]:
w1 = w(psr)

In [42]:
w1.params

dict_values(["J2222-0137_efac":Uniform(0.5,1.5)])

In [50]:
w1.ndiag({'J2222-0137_efac': 1.0})[0]

3.7423676303999994e-10

In [51]:
w1.ndiag({'J2222-0137_efac': 1.5})[0]

8.4203271683999982e-10

In [46]:
x = MeasurementNoise(efac=Normal(1,0.5))

In [47]:
x1 = x(psr)

In [48]:
x1.params

dict_values([<__main__.Normal.<locals>.Normal object at 0x1164d1fd0>])

In [132]:
def Function(f,**kwargs):
    class Function(object):
        def __init__(self,prefix):
            self._params = {kw: arg(prefix + '_' + kw) for kw,arg in kwargs.items()}
        
        # params could also be a standard argument here,
        # but by defining it as ** we allow multiple positional arguments
        def __call__(self,*args,**params):
            pardict = {kw: params[par.name] for kw,par in self._params.items()
                                            if par.name in params}
            return f(*args,**pardict)
        
        @property
        def params(self):
            return self._params.values()
    
    return Function

In [133]:
year = 365.25 * 24 * 3600

# unfortunately lambdas are not acceptable here because of the way we call them
# a def is not too bad though
def powerlaw(f,log10_A=-16,gamma=5):
    return (10**log10_A) * (f*year)**(-gamma)

PowerLaw = Function(powerlaw,log10_A=Uniform(-18,-12),gamma=Uniform(1,7))

In [134]:
PowerLaw

__main__.Function.<locals>.Function

In [135]:
pw = PowerLaw('J2222-0137')

In [136]:
pw.params

dict_values(["J2222-0137_gamma":Uniform(1,7), "J2222-0137_log10_A":Uniform(-18,-12)])

In [137]:
pw._params

{'gamma': "J2222-0137_gamma":Uniform(1,7),
 'log10_A': "J2222-0137_log10_A":Uniform(-18,-12)}

In [144]:
pw(1e-7,**{'J2222-0137_gamma': 5,"J2222-0137_log10_A": -16})

3.1950684338930047e-19

In [145]:
# defaults are respected
pw(1e-7,**{"J2222-0137_log10_A": -16})

3.1950684338930047e-19

In [165]:
# start with marginalized only... my actual matrices may be wrong, since I did them very fast

def FourierBasisGP(spectrum=None,components=20):
    class FourierBasisGP(Signal):
        def __init__(self,psr):
            self._psr = psr

            self._spectrum = spectrum(psr.name)
            self._params = self._spectrum._params

            self._toas = psr.stoas

            # should use a common epoch? no subtraction at all?
            self._t = 86400.0 * (self._toas - np.min(self._toas))
            self._T = np.max(self._toas) - np.min(self._toas)
            
            self._f = np.arange(1,components+1) / self._T
            
            self._f2 = np.zeros(2*len(self._f),'d')
            self._f2[0::2] = self._f2[1::2] = self._f
            
            self._F = np.zeros((len(self._t),2*len(self._f)),'d')
            for i in range(components):
                self._F[:,2*i]   = np.cos(2*math.pi*self._f[i]*self._t)
                self._F[:,2*i+1] = np.sin(2*math.pi*self._f[i]*self._t)
                    
        def Fmat(self,params=None):
            return self._F
        
        # maybe we only need diagonal Phi?
        def Phivec(self,params):
            return self._spectrum(self._f2,**params)
    
    return FourierBasisGP

In [171]:
f = FourierBasisGP(spectrum=PowerLaw)

In [167]:
f1 = f(psr)

In [168]:
f1.params

dict_values(["J2222-0137_gamma":Uniform(1,7), "J2222-0137_log10_A":Uniform(-18,-12)])

In [169]:
f1._f

array([ 0.0010900472,  0.0021800945,  0.0032701417,  0.004360189,
        0.0054502362,  0.0065402835,  0.0076303307,  0.008720378,
        0.0098104252,  0.010900472,  0.01199052,  0.013080567,
        0.014170614,  0.015260661,  0.016350709,  0.017440756,
        0.018530803,  0.01962085,  0.020710898,  0.021800945], dtype=float128)

In [170]:
f1.Phivec({"J2222-0137_gamma": 5, "J2222-0137_log10_A": -16})

array([  2.07612527e-39,   2.07612527e-39,   6.48789146e-41,
         6.48789146e-41,   8.54372538e-42,   8.54372538e-42,
         2.02746608e-42,   2.02746608e-42,   6.64360086e-43,
         6.64360086e-43,   2.66991418e-43,   2.66991418e-43,
         1.23527415e-43,   1.23527415e-43,   6.33583151e-44,
         6.33583151e-44,   3.51593637e-44,   3.51593637e-44,
         2.07612527e-44,   2.07612527e-44,   1.28911045e-44,
         1.28911045e-44,   8.34348182e-45,   8.34348182e-45,
         5.59160897e-45,   5.59160897e-45,   3.86023173e-45,
         3.86023173e-45,   2.73399212e-45,   2.73399212e-45,
         1.97994735e-45,   1.97994735e-45,   1.46220730e-45,
         1.46220730e-45,   1.09873012e-45,   1.09873012e-45,
         8.38466179e-46,   8.38466179e-46,   6.48789146e-46,
         6.48789146e-46])